In [1]:
import numpy as np
import scipy.linalg as sc
import warnings
warnings.filterwarnings("ignore")

Résolution de Ax=b avec A une matrice carrée n×n et b∈Rn

    Résolution d'un système linéaire triangulaire
        supérieur
        inférieur
    Méthode de Gauss
        Méthode de triangulation de Gauss
        Appel à la résolution
    Résolution par décomposition LU
        Décomposition LU de la matrice A
        Résolution du système initial par la résolution de deux système triangulaires (supérieur et inférieur)
    Méthodes itératives
    Si vous avez le temps, vous écrirez les méthodes par calcul des éléments de xi

    et également l'écriture matricielle.
        Méthode de Jabobi
        Méthode de Gauss-Seidel
        Méthode de relaxation

Implantez la méthode du calcul de l'inverse d'une matrice carrée A
par "double triangulation", puis division des lignes par les éléments diagonaux.

# Outils TP5

In [2]:
def genere_Tij(n,i,j):
    A=np.eye(n)
    A[[i,j]]=A[[j,i]]
    return A

In [3]:
def genere_Lrs_λ(n,r,s,λ):
    M=np.eye(n)
    M[r,s]=λ
    return M

In [4]:
def generation_M_second_way(A,r):
    n=len(A)
    M=np.eye(n)
    Minverse=np.eye(n)
    for i in range(r+1,n):
        M=M @ genere_Lrs_λ(n,i,r,-A[i,r]/A[r,r])
        Minverse=Minverse @ genere_Lrs_λ(n,i,r,A[i,r]/A[r,r])
    return M,Minverse

## 1) Résolution de Ax=b avec A une matrice carrée n×n et b∈Rn

## Résolution d'un système linéaire triangulaire

In [5]:
def resol_superieur(A,b):
    n=A.shape[0]
    x=np.zeros(n)
    n=n-1
    x[n]=b[n]/A[n,n]
    for i in range(n-1,-1,-1):
        x[i]=(b[i]-sum(A[i,i+1:]*x[i+1:]))/A[i,i]
    return x

In [6]:
A=np.random.randint(1,6,(3,3))

In [7]:
l,u=sc.decomp_lu.lu(A)[1:3]
u

array([[5.    , 4.    , 1.    ],
       [0.    , 3.2   , 3.8   ],
       [0.    , 0.    , 3.0625]])

In [8]:
res=resol_superieur(u,np.array([1,1,-2]))
res

array([-0.53979592,  1.0880102 , -0.65306122])

In [9]:
u@res

array([ 1.,  1., -2.])

In [10]:
def resol_inferieur(A,b):
    n=A.shape[0]
    x=np.zeros(n)
    x[0]=b[0]/A[0,0]
    for i in range(1,n):
        x[i]=(b[i]-sum(A[i,0:i]*x[0:i]))/A[i,i]
    return x

In [11]:
res=resol_inferieur(l,np.array([1,4,-1]))
res

array([1.    , 3.8   , 0.0625])

In [12]:
l@res

array([ 1.,  4., -1.])

# Méthode de Gauss

In [13]:
def triangulation_de_Gauss(A,b):
    A=np.hstack((A,np.array([[i] for i in b])))
    for i in range(A.shape[1]-2):
        if A[:,i].all()==0:
            continue
        maxx=np.argmax(np.abs(A[i+1:,i]))+i+1
        if np.abs(A[maxx,i])>A[i,i]:
             A=genere_Tij(A.shape[0],maxx,i)@A
        A=generation_M_second_way(A,i)[0]@A
    b=A[:,A.shape[1]-1]
    A=A[:,0:A.shape[1]-1]
    return A,b

In [14]:
triangulation_de_Gauss(A,np.array([1,2,3]))

(array([[5.    , 4.    , 1.    ],
        [0.    , 3.2   , 3.8   ],
        [0.    , 0.    , 3.0625]]),
 array([1.   , 2.8  , 2.625]))

In [15]:
def resolution_de_Gauss(A,b):
    return resol_superieur(A,b)

In [16]:
a,c=triangulation_de_Gauss(A,np.array([1,2,3]))
resolution_de_Gauss(a,c)

array([ 0.14285714, -0.14285714,  0.85714286])

In [17]:
A@resolution_de_Gauss(a,c)

array([1., 2., 3.])

# Decomposition LU

In [18]:
def resolut_decompostion_LU(A,b):
    L=np.eye(A.shape[0])
    for i in range(A.shape[1]-1):
        u,l=generation_M_second_way(A,i)
        A=u@A
        L=L@l
        y=resol_inferieur(L,b)
        x=resol_superieur(A,y)
    return x    

In [19]:
res=resolut_decompostion_LU(A,np.array([1,2,3]))
res

array([ 0.14285714, -0.14285714,  0.85714286])

In [20]:
A@res

array([1., 2., 3.])

# Methode Iterative

In [56]:
def jacobi(A,b,prec):
    x_prec=np.zeros(len(b))
    x_suiv=np.array([(b[i]-sum([A[i,j]*x_prec[j] for j in range(0,len(A)) if i !=j]))/A[i,i] for i in range(len(b))])
    x=x_suiv-x_prec
    x_prec=x_suiv
    max_iter=0 #si la methode ne converge pas
    while np.sqrt(np.abs(x.T@x))>prec and max_iter<1000:
         x_suiv=np.array([(b[i]-sum([A[i,j]*x_prec[j] for j in range(0,len(A)) if i !=j]))/A[i,i] for i in range(len(b))])
         x=x_suiv-x_prec
         x_prec=x_suiv
         max_iter=max_iter+1
    return x_suiv

In [57]:
mat=np.ones((3,3))
for i in range(len(mat)):
    mat[i,i]=5+i

s=jacobi(mat,np.array([1,4,3]),0.01)
s

array([0.01199093, 0.60961451, 0.34147068])

In [58]:
mat@s

array([1.01103984, 4.01114869, 3.01190023])

# Les resultats de ma methode de jacobi me semble inconherents quelque soit la precision alors que je ne vois pas d'errer sur l'implementation de la methode

In [62]:
def gauss_sedal_and_relaxation(A,b,prec,alpha=1):
    """
      Si alpha==1
         la methode resoud l'equation avec la methode de gauss_sedal si apla=1
      Sino
         il le resoud avec la methode de la relaxation
      
    """
    x_prec=np.zeros(len(b))
    x_suiv=np.zeros(len(b))
    x_temp=np.zeros(len(b))
    for i in range(len(b)):
        x_suiv[i]=alpha*(b[i]-sum([A[i,j]*x_prec[j] if i<j else A[i,j]*x_temp[j] for j in range(0,len(A)) if i !=j]))/A[i,i]+(1-alpha)*x_prec[i]
        x_temp[i]=x_suiv[i]
        x=x_suiv-x_prec
        x_prec=x_suiv
    max_iter=0 #si la methode ne converge pas
    x=x_suiv-x_prec
    while(np.sqrt(np.abs(x.T@x))>prec) or max_iter<1000:
        for i in range(len(b)):
            x_suiv[i]=alpha*(b[i]-sum([A[i,j]*x_prec[j] if i<j else A[i,j]*x_temp[j] for j in range(0,len(A)) if i !=j]))/A[i,i]+(1-alpha)*x_prec[i]
            x_temp[i]=x_suiv[i]
            x=x_suiv-x_prec
            x_prec=x_suiv
            max_iter=max_iter+1
    return x_suiv

In [63]:
s=gauss_sedal_and_relaxation(mat,np.array([1,0,3]),0.01)
s

array([ 0.13402062, -0.09278351,  0.42268041])

In [64]:
mat@s

array([1., 0., 3.])

# Implantez la méthode du calcul de l'inverse d'une matrice carrée A par "double triangulation", puis division des lignes par les éléments diagonaux.

In [140]:
def genere_sup(A):
     Msup=[]
     for i in range(A.shape[1]-1):
        u=generation_M_second_way(A,i)[0]
        A=u@A
        Msup.append(u)
     return A,Msup

In [142]:
genere_sup(A)[0]

array([[ 5. ,  4. ,  1. ],
       [ 0. , -1.4,  1.4],
       [ 0. ,  0. ,  7. ]])

In [98]:
def generation_M_inf(A,r):
    n=len(A)
    M=np.eye(n)
    for i in range(0,r):
        M=M @ genere_Lrs_λ(n,i,r,-A[i,r]/A[r,r])
    return M

In [156]:
def genere_inf(A):
     Minf=[]
     for i in range(len(A)-1,0,-1):
        u=generation_M_inf(A,i)
        A=u@A
        Minf.append(u)
     return A,Minf

In [244]:
def calcul_inverse(A):
    Minf=genere_inf(A)[1]
    Msup=genere_sup(A)[1]
    M1=np.eye(len(A))
    M2=np.eye(len(A))
    diag=genere_sup(genere_inf(A)[0])[0]
    diag=np.where(np.abs(diag)<0.0001,0,diag) #precision de la macine
    for i in range(len(diag)):
        diag[i,i]= 1/diag[i,i] # inverse
    for i in range(len(Minf)-1,-1,-1):
         M1=M1@Minf[i]
         M2=M2@Msup[i]
    return diag@M2@M1

In [245]:
calcul_inverse(A)@A

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.85000000e+00,  1.00000000e+00, -4.44089210e-16],
       [-3.13392857e+00,  4.28571429e-01,  1.00000000e+00]])

In [234]:
genere_inf(A)[0]

array([[12.25,  0.  ,  0.  ],
       [ 2.5 , -1.  ,  0.  ],
       [ 1.  ,  4.  ,  4.  ]])

In [208]:
np.linalg.inv(A)

array([[ 0.08163265,  0.24489796, -0.14285714],
       [ 0.20408163, -0.3877551 ,  0.14285714],
       [-0.2244898 ,  0.32653061,  0.14285714]])